In [1]:
from openprompt.prompts import PtuningTemplate
from openprompt.plms import MLMTokenizerWrapper
from datasets import load_from_disk
from openprompt.prompts import ManualVerbalizer
from openprompt import PromptForClassification
from openprompt.data_utils import InputExample
from random import shuffle
from transformers import RobertaTokenizer,RobertaForMaskedLM
from openprompt.pipeline_base import PromptDataLoader
from transformers.optimization import get_linear_schedule_with_warmup
from transformers import AdamW
import torch
from datasets import load_from_disk
from tqdm import tqdm
import json
from openprompt.data_utils import FewShotSampler


dataset_path = "/lustre/work/client/users/minhos/cache/datasets/p3_rotten_tomato"
raw_dataset = load_from_disk(dataset_path)

label_map = {"positive": 0,"negative": 1}

dataset = {}
for split in ['train', 'validation']:
    dataset[split] = []
    raw_dataset[split] = raw_dataset[split].shuffle(seed=42).select(range(1000))
    for idx, data in enumerate(raw_dataset[split]):
        label_text = data["targets_pretokenized"].strip().lower()
        label_numeric = label_map.get(label_text, -1)
        input_example = InputExample(text_a = data['inputs_pretokenized'], guid=idx, label=label_numeric)
        dataset[split].append(input_example)
print(dataset['train'][0])
print(type(dataset['train'][0]))

sampler = FewShotSampler(num_examples_per_label=30)
fewshot_data = sampler(dataset['train'], seed=42)

# Load the RoBERTa model and tokenizer
roberta_path = "/lustre/work/client/users/minhos/models_for_supercomputer/facebook_roberta-base"
model = RobertaForMaskedLM.from_pretrained(roberta_path)
tokenizer = RobertaTokenizer.from_pretrained(roberta_path)
tokenizer.pad_token = tokenizer.eos_token 


# Logging setup
log_file = "prefix_tuning_results_roberta_6.json"
results = []

# Hyperparameter search ranges
learning_rates = [1e-4] # 1e-4, 3e-5
num_soft_tokens = [100] # 10, 50, 100
warmup_steps = [20, 25] # 10, 20, 25


for lr in learning_rates:
    for tokens in num_soft_tokens:
        for warmup in warmup_steps:
            print(f"Testing: LR={lr}, Soft Tokens={tokens}, Warm-Up Steps={warmup}")

            model = RobertaForMaskedLM.from_pretrained(roberta_path)
            tokenizer = RobertaTokenizer.from_pretrained(roberta_path)
            tokenizer.pad_token = tokenizer.eos_token 


            template = PtuningTemplate(
                model=model,
                tokenizer=tokenizer,
                text='{"placeholder":"text_a"} {"mask"}',
                prompt_encoder_type="lstm",  # Choose between "lstm" or "mlp"
            )

            verbalizer = ManualVerbalizer(
                tokenizer=tokenizer, 
                num_classes=2,  # Example: binary classification
                label_words=[["positive", "good", "excellent", "wonderful"], ["negative", "bad", "horrible", "terrible"]],
                classes= [0, 1]
            )

            prompt_model = PromptForClassification(
                plm=model,
                template=template,
                verbalizer=verbalizer,
                freeze_plm=True,  # Set to False if you want to fine-tune RoBERTa
            )


            train_dataloader = PromptDataLoader(
                dataset=fewshot_data,
                template=template,
                tokenizer=tokenizer,
                tokenizer_wrapper_class=MLMTokenizerWrapper,
                max_seq_length=480, decoder_max_length=3,
                batch_size=5,shuffle=True, teacher_forcing=False, predict_eos_token=False,
                truncate_method="tail",
            )
            
            validation_dataloader = PromptDataLoader(
                dataset=dataset["validation"],
                template=template,
                tokenizer=tokenizer,
                tokenizer_wrapper_class=MLMTokenizerWrapper,
                decoder_max_length=3,
                batch_size=5,shuffle=False, teacher_forcing=False, predict_eos_token=False,
                truncate_method="tail",
            )

            # Unfreeze parameters and prepare grouped parameters for the optimizer
            optimizer_grouped_parameters = []

            for name, param in prompt_model.named_parameters():
                if not param.requires_grad and param.dtype in [torch.float32, torch.float64, torch.complex64, torch.complex128]:
                    param.requires_grad = True   
                optimizer_grouped_parameters.append({'params': param})

            # Define optimizer for the prefix tuning parameters
            optimizer = AdamW(optimizer_grouped_parameters, lr=lr)
            loss_func = torch.nn.CrossEntropyLoss()

            # Define a learning rate scheduler
            scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=10, num_training_steps=1000)
            # Ensure model is in training mode
            prompt_model.train()

            # Training loop
            num_epochs = 10
            gradient_accumulation_steps = 1

            for epoch in range(num_epochs):
                print(f"Epoch {epoch + 1}/{num_epochs}")
                total_loss = 0
                pbar = tqdm(train_dataloader, desc="Training")
                
                for step, inputs in enumerate(pbar):
                    logits = prompt_model(inputs)
                    labels = inputs['label']  # Ground-truth labels
                    
                    # Compute loss
                    loss = loss_func(logits, labels)
                    loss.backward()  # Backpropagation
                    
                    # Optimizer step
                    if (step + 1) % gradient_accumulation_steps == 0:
                        optimizer.step()
                        scheduler.step()
                        optimizer.zero_grad()
                    
                    total_loss += loss.item()
                    pbar.set_postfix({"loss": total_loss / (step + 1)})
                    
            def evaluate(prompt_model, dataloader):
                prompt_model.eval()  # Set the model to evaluation mode
                total, correct = 0, 0
                
                with torch.no_grad():
                    for inputs in dataloader:
                        logits = prompt_model(inputs)
                        preds = torch.argmax(logits, dim=-1)
                        labels = inputs['label']
                        
                        total += len(labels)
                        correct += (preds == labels).sum().item()
                
                accuracy = correct / total
                return accuracy
                
            # Validation after each epoch
            val_accuracy = evaluate(prompt_model, validation_dataloader)
            print(f"Validation Accuracy after Epoch {epoch + 1}: {val_accuracy:.4f}")
            # Log results
            result = {
                "learning_rate": lr,
                "num_soft_tokens": tokens,
                "warmup_steps": warmup,
                "final_loss": total_loss / (10 * len(train_dataloader)),
                "accuracy": val_accuracy
            }
            results.append(result)

            # Save intermediate results
            with open(log_file, "w") as f:
                json.dump(results, f, indent=4)
            
print("Tuning complete. Results saved to", log_file)

/lustre/work/client/users/minhos/.conda/envs/prompt_learning_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/lustre/work/client/users/minhos/.conda/envs/prompt_learning_env/lib/python3.10/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


{
  "guid": 0,
  "label": 1,
  "meta": {},
  "text_a": ". . . plays like somebody spliced random moments of a chris rock routine into what is otherwise a cliche-riddled but self-serious spy thriller . What is the sentiment expressed by the reviewer for the movie? ",
  "text_b": "",
  "tgt_text": null
}

<class 'openprompt.data_utils.utils.InputExample'>


/lustre/work/client/users/minhos/.conda/envs/prompt_learning_env/lib/python3.10/site-packages/transformers/modeling_utils.py:399: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

Testing: LR=0.0001, Soft Tokens=100, Warm-Up Steps=20


tokenizing: 60it [00:00, 383.76it/s]
tokenizing: 1000it [00:01, 922.41it/s]
/lustre/work/client/users/minhos/.conda/envs/prompt_learning_env/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


...

Epoch 9/10


Training: 100%|██████████| 12/12 [01:00<00:00,  5.07s/it, loss=0.31] 


Epoch 10/10


Training: 100%|██████████| 12/12 [01:08<00:00,  5.75s/it, loss=0.275]


Validation Accuracy after Epoch 10: 0.6110
Tuning complete. Results saved to prefix_tuning_results_roberta_6.json
